In [1]:
import tensorflow as tf
tf.__version__

'2.1.0'

In [2]:
import os
import sys
import random
import warnings

import numpy as np
import pandas as pd
# import matplotlib
# import matplotlib.pyplot as plt
# plt.style.use("ggplot")

# # matplotlib.use('agg')
import matplotlib.pyplot as plt
%matplotlib inline


from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

import cv2
print("Imported Tensorflow")

Using TensorFlow backend.


Imported Tensorflow


In [3]:
tf.__version__

'2.1.0'

In [4]:
# Set some parameters
IMG_WIDTH = 512
IMG_HEIGHT = 512
IMG_CHANNELS = 1
TRAIN_PATH = '../input/lungsenhanced/LungsEnhanced/'
TEST_PATH = '../input/unetlungsdata/data/test/'

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 2040
random.seed = seed
np.random.seed = seed

print("Imported all the dependencies")

Imported all the dependencies


In [5]:
#print(os.listdir("../input/weights/best.hdf5"))

FileNotFoundError: [Errno 2] No such file or directory: '../input/weights/best.hdf5'

In [ ]:

train_ids = next(os.walk(TRAIN_PATH))[1]
empty = []
for train_id in train_ids:
    if os.path.exists(TRAIN_PATH+train_id+'/images'):
        continue
    else:
        empty.append(train_id)
train_ids = list(set(train_ids) - set(empty))
print(len(train_ids))
train_ids_1 = train_ids[:len(train_ids)//2]
train_ids_2 = train_ids[len(train_ids)//2:]

In [ ]:
# Get train and test IDs
#train_ids = next(os.walk(TRAIN_PATH))[1]

# Get and resize train images and masks
X_train = np.zeros((len(train_ids_2), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(train_ids_2), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)

print("X_train",X_train.shape)
print("Y_train",Y_train.shape)
print('Getting and resizing train images and masks ... ')
sys.stdout.flush()
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
for n, id_ in tqdm(enumerate(train_ids_2), total=len(train_ids_2)):
	path = TRAIN_PATH + id_
	img = imread(path + '/images/' + id_ + '.png')
	img = resize(img, (IMG_HEIGHT, IMG_WIDTH, 1), mode='constant', preserve_range=True)
	X_train[n] = img
	mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
	for mask_file in next(os.walk(path + '/masks/'))[2]:
		mask_ = imread(path + '/masks/' + mask_file)
		mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant',preserve_range=True), axis=-1)
		mask = np.maximum(mask, mask_)
	Y_train[n] = mask

print('Done!')

In [ ]:
# Check if training data looks all right
ix = random.randint(0, len(train_ids_1))

imshow(np.squeeze(X_train[ix]))
plt.show();
plt.savefig('img.png')

imshow(np.squeeze(Y_train[ix]))
plt.show();
plt.savefig('mask.png')

In [ ]:
# Define IoU metric
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.dtypes.cast(y_pred > t,tf.int32)
        score, up_opt = tf.metrics.MeanIoU(num_classes=2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [ ]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())

In [ ]:
from keras import backend as K
def iou_coef(y_true, y_pred, smooth=1):
  intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2])
  union = K.sum(y_true,[1,2])+K.sum(y_pred,[1,2])-intersection
  iou = K.mean((intersection ) / (union), axis=0)
  return iou

In [ ]:
import keras.backend as K
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1,2])
    union = K.sum(y_true, axis=[1,2]) + K.sum(y_pred, axis=[1,2])
    return K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)
def dice_p_bce(in_gt, in_pred):
    return 0.0*binary_crossentropy(in_gt, in_pred) - dice_coef(in_gt, in_pred)
def true_positive_rate(y_true, y_pred):
    return K.sum(K.flatten(y_true)*K.flatten(K.round(y_pred)))/K.sum(y_true)
#seg_model.load_weights('../input/weights/seg_model_weights.best.hdf5')
#seg_model.compile(optimizer=Adam(1e-4, decay=1e-6), loss=dice_p_bce, metrics=[dice_coef, 'binary_accuracy', true_positive_rate])

In [ ]:
# X_train = X_train[:15000]
Y_train = Y_train[:15000]

In [ ]:
from keras.layers import Input, Activation, Conv2D, MaxPool2D, UpSampling2D, Dropout, concatenate, BatchNormalization, Cropping2D, ZeroPadding2D, SpatialDropout2D
from keras.layers import Conv2DTranspose, Dropout, GaussianNoise
from keras.models import Model
from keras import backend as K

def up_scale(in_layer):
    filt_count = in_layer._keras_shape[-1]
    return Conv2DTranspose(filt_count, kernel_size = (2,2), strides = (2,2), padding = 'same')(in_layer)
def up_scale_fancy(in_layer):
    return UpSampling2D(size=(2,2))(in_layer)

input_layer = Input(shape=X_train.shape[1:])
sp_layer = GaussianNoise(0.1)(input_layer)
bn_layer = BatchNormalization()(sp_layer)
c1 = Conv2D(filters=8, kernel_size=(5,5), activation='relu', padding='same')(bn_layer)
l = MaxPool2D(strides=(2,2))(c1)
c2 = Conv2D(filters=16, kernel_size=(3,3), activation='relu', padding='same')(l)
l = MaxPool2D(strides=(2,2))(c2)
c3 = Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same')(l)

l = MaxPool2D(strides=(2,2))(c3)
c4 = Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same')(l)

l = SpatialDropout2D(0.25)(c4)
dil_layers = [l]
for i in [2, 4, 6, 8, 12, 18, 24]:
    dil_layers += [Conv2D(16,
                          kernel_size = (3, 3), 
                          dilation_rate = (i, i), 
                          padding = 'same',
                         activation = 'relu')(l)]
l = concatenate(dil_layers)


l = SpatialDropout2D(0.2)(concatenate([up_scale(l), c3], axis=-1))
l = Conv2D(filters=128, kernel_size=(2,2), activation='linear', padding='same')(l)
l = BatchNormalization()(l)
l = Activation('relu')(l)
u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (l)
l = Conv2D(filters=96, kernel_size=(2,2), activation='relu', padding='same')(u9)
l = Conv2D(filters=32, kernel_size=(2,2), activation='relu', padding='same')(l)
u10 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (l)
l = Conv2D(filters=16, kernel_size=(2,2), activation='linear', padding='same')(u10)

l = Cropping2D((4,4))(l)
l = BatchNormalization()(l)
l = Activation('relu')(l)

l = Conv2D(filters=1, kernel_size=(1,1), activation='sigmoid')(l)
output_layer = ZeroPadding2D((4,4))(l)

seg_model = Model(input_layer, output_layer)
seg_model.load_weights("../input/weights3/best.hdf5")
seg_model.compile(optimizer=Adam(1e-4, decay=1e-6), loss=dice_p_bce, metrics=[tf.metrics.MeanIoU(num_classes=2), dice_coef, 'binary_accuracy', true_positive_rate])
seg_model.summary()

In [ ]:
import keras.backend as K
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1,2])
    union = K.sum(y_true, axis=[1,2]) + K.sum(y_pred, axis=[1,2])
    return K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)
def dice_p_bce(in_gt, in_pred):
    return 0.0*binary_crossentropy(in_gt, in_pred) - dice_coef(in_gt, in_pred)
def true_positive_rate(y_true, y_pred):
    return K.sum(K.flatten(y_true)*K.flatten(K.round(y_pred)))/K.sum(y_true)
seg_model.compile(optimizer=Adam(1e-4, decay=1e-6), loss=dice_p_bce, metrics=[tf.metrics.MeanIoU(num_classes=2),dice_coef, 'binary_accuracy', true_positive_rate])

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
weight_path="{}_weights.best.hdf5".format('seg_model')

checkpoint = ModelCheckpoint(weight_path, monitor='val_dice_coef', verbose=1, 
                             save_best_only=True, mode='max', save_weights_only = True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_dice_coef', factor=0.5, 
                                   patience=3, 
                                   verbose=1, mode='max', epsilon=0.0001, cooldown=2, min_lr=1e-6)
early = EarlyStopping(monitor="mean_io_u_", 
                      mode="max", 
                      patience=15) # probably needs to be more patient, but kaggle time is limited
callbacks_list = [checkpoint, early, reduceLROnPlat]

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
dg_args = dict(featurewise_center = False, 
                  samplewise_center = False,
                  rotation_range = 7.5, 
                  width_shift_range = 0.02, 
                  height_shift_range = 0.02, 
                  shear_range = 0.01,
                  zoom_range = [0.9, 1.25],  
                  brightness_range = [0.9, 1.1],
                  horizontal_flip = False, 
                  vertical_flip = False,
                  fill_mode = 'nearest',
                   data_format = 'channels_last')

image_gen = ImageDataGenerator(**dg_args)
dg_args.pop('brightness_range')
label_gen = ImageDataGenerator(**dg_args)
# def train_gen(batch_size = 16, seed = None):
#     np.random.seed(seed if seed is not None else np.random.choice(range(9999)))
#     while True:
#         seed = np.random.choice(range(9999))
#         # keep the seeds syncronized otherwise the augmentation to the images is different from the masks
#         batch_count = X_train.shape[0]//batch_size
#         batch_id = np.random.permutation(range(0, X_train.shape[0]-batch_size, batch_size))
#         for c_idx in batch_id:
#             g_x = image_gen.flow(X_train[c_idx:(c_idx+batch_size)], batch_size = batch_size, seed = seed, shuffle=True)
#             g_y = label_gen.flow(Y_train[c_idx:(c_idx+batch_size)], batch_size = batch_size, seed = seed, shuffle=True)
#             yield next(g_x)/255.0, next(g_y)

In [ ]:
batch_size = 32
loss_history = [seg_model.fit(X_train, Y_train, validation_split=0.1, batch_size=32, epochs=50,callbacks=callbacks_list)]
#                 fit_generator(image_gen.flow(X_train, Y_train, batch_size=batch_size),
#                              steps_per_epoch=X_train.shape[0]//batch_size, 
#                              epochs=40, 
#                              validation_split=0.1,
#                              callbacks=callbacks_list,
#                             workers=2)]


In [ ]:
seg_model.save('../working/bestweights.hdf5')

In [ ]:
fig, m_axs = plt.subplots(4,3, figsize = (20, 20))
[c_ax.axis('off') for c_ax in m_axs.flatten()]
for idx, (ax1, ax2, ax3) in enumerate(m_axs):
    ix = X_train[idx:idx+1]
    iy = Y_train[idx:idx+idx+1]
    p_image = seg_model.predict(ix)
    ax1.imshow(ix[0,:,:,0], cmap = 'gray')
    ax1.set_title('Input Image')
    ax2.imshow(iy[0,:,:,0], vmin = 0, vmax = 1, cmap = 'bone_r' )
    ax2.set_title('Ground Truth')
    ax3.imshow(p_image[0,:,:,0], vmin = 0, vmax = 1, cmap = 'viridis' )
    ax3.set_title('Prediction')

In [ ]:
# # Build U-Net model
# import keras
# inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
# s = Lambda(lambda x: x / 255) (inputs)

# c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (s)
# c1 = Dropout(0.1) (c1)
# c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
# p1 = MaxPooling2D((2, 2)) (c1)

# c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
# c2 = Dropout(0.1) (c2)
# c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
# p2 = MaxPooling2D((2, 2)) (c2)

# c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
# c3 = Dropout(0.2) (c3)
# c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
# p3 = MaxPooling2D((2, 2)) (c3)

# c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
# c4 = Dropout(0.2) (c4)
# c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
# p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

# c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
# c5 = Dropout(0.3) (c5)
# c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

# u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
# u6 = concatenate([u6, c4])
# c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
# c6 = Dropout(0.2) (c6)
# c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

# u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
# u7 = concatenate([u7, c3])
# c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
# c7 = Dropout(0.2) (c7)
# c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

# u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
# u8 = concatenate([u8, c2])
# c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
# c8 = Dropout(0.1) (c8)
# c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

# u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
# u9 = concatenate([u9, c1], axis=3)
# c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
# c9 = Dropout(0.1) (c9)
# c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

# outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

# model = Model(inputs=[inputs], outputs=[outputs])
# model.load_weights('../input/bestweights/seg_model_weights.best.hdf5')
# model.compile(optimizer='adam', loss=[iou_coef], metrics=[tf.metrics.MeanIoU(num_classes=2), dice_coef, 'binary_accuracy', true_positive_rate])
# model.summary()
# print("The model is defined")

In [ ]:
# Fit model
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint('model-dsbowl2018-1.h5', verbose=1, save_best_only=True)
results = model.fit(X_train, Y_train, validation_split=0.1, batch_size=30, epochs=5)


In [ ]:

preds_train = model.predict(X_train[:int(X_train.shape[0]*0.9)], verbose=1)
preds_val = model.predict(X_train[int(X_train.shape[0]*0.9):], verbose=1)
preds_test = model.predict(X_test, verbose=1)

# Threshold predictions

preds_train_t = (preds_train > 0.1).astype(np.uint8)
preds_val_t = (preds_val > 0.1).astype(np.uint8)
preds_test_t = (preds_test > 0.1).astype(np.uint8)

In [ ]:

# Create list of upsampled test masks
preds_test_upsampled = []
for i in range(len(preds_test)):
    preds_test_upsampled.append(resize(np.squeeze(preds_test[i]), 
                                       (sizes_test[i][0], sizes_test[i][1]), 
                                       mode='constant', preserve_range=True))


In [ ]:


# Perform a sanity check on some random training samples
ix = random.randint(0, len(preds_train_t))
imshow(np.squeeze(X_train[ix]))
plt.show();
plt.savefig('img.png')
imshow(np.squeeze(Y_train[ix]))
plt.show();
plt.savefig('gt.png')
imshow(np.squeeze(preds_train[ix]))
plt.show();
plt.savefig('predict.png')

In [ ]:
preds_train_t[ix].shape

In [ ]:
from zf_unet_224_model import ZF_UNET_224, dice_coef_loss, dice_coef
from keras.optimizers import Adam

model = ZF_UNET_224(weights='generator')
optim = Adam()
model.compile(optimizer=optim, loss=dice_coef_loss, metrics=[dice_coef])

results = model.fit(X_train, Y_train, validation_split=0.1, batch_size=30, epochs=5)